In [1]:
import sys
import os
import time
import warnings
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Union, Any
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.cuda.amp import autocast
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle
import json
from scipy.ndimage import zoom, gaussian_filter
from skimage.measure import label
from skimage.morphology import remove_small_objects, binary_closing

sys.path.append(r"c:\Users\surya\NueroVisionAI\core\3D architecture")

from utils_3d import Config, setup_logging, get_device, timer, format_time, load_nifti, save_nifti
from model_3d import UNet3D, create_unet3d
from dataset_3d import BraTSDataset
from preprocess_3d import VolumePreprocessor, preprocess_for_inference
from postprocess_3d import PostProcessor3D
from visualize_3d import VolumeVisualizer
from inference_3d import ModelInference, InferenceResult
from metrics_3d import SegmentationMetrics

os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'
warnings.filterwarnings('ignore')

print("🧠" * 50)
print("3D BRAIN TUMOR SEGMENTATION INFERENCE")
print("PRODUCTION-READY INFERENCE PIPELINE")
print("🧠" * 50)

device = get_device("auto")
print(f"\n🔥 GPU STATUS:")
print(f"├── Device: {device}")
if torch.cuda.is_available():
    print(f"├── GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"├── CUDA Version: {torch.version.cuda}")
    print(f"├── PyTorch Version: {torch.__version__}")
    print(f"└── Mixed Precision: ENABLED ✅")
else:
    print(f"└── CPU Mode: Enabled")

config = Config()
config.input_size = (64, 64, 64)
config.model_name = "light"
config.base_filters = 16
config.depth = 3
config.device = "auto"
config.mixed_precision = True
config.data_dir = r"c:\Users\surya\NueroVisionAI\data\BrainWithTumor"
config.output_dir = r"c:\Users\surya\NueroVisionAI\outputs\3d"
config.checkpoint_dir = r"c:\Users\surya\NueroVisionAI\checkpoints\3d"

use_tta = True
use_uncertainty = True
confidence_threshold = 0.5
min_tumor_size = 50
model_path = r"c:\Users\surya\NueroVisionAI\outputs\3d\unet3d_model.pkl"
checkpoint_path = r"c:\Users\surya\NueroVisionAI\checkpoints\3d\best_model.pth"

print(f"\n🔧 INFERENCE CONFIGURATION:")
print(f"├── Model: {config.model_name} 3D U-Net")
print(f"├── Volume Size: {config.input_size}")
print(f"├── Mixed Precision: {'✅ ENABLED' if config.mixed_precision else '❌ DISABLED'}")
print(f"├── Test-Time Augmentation: {'✅ ENABLED' if use_tta else '❌ DISABLED'}")
print(f"├── Uncertainty Estimation: {'✅ ENABLED' if use_uncertainty else '❌ DISABLED'}")
print(f"└── Confidence Threshold: {confidence_threshold}")

Path(config.output_dir).mkdir(parents=True, exist_ok=True)

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠
3D BRAIN TUMOR SEGMENTATION INFERENCE
PRODUCTION-READY INFERENCE PIPELINE
🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

🔥 GPU STATUS:
├── Device: cuda
├── GPU Name: NVIDIA GeForce RTX 4060 Laptop GPU
├── CUDA Version: 11.8
├── PyTorch Version: 2.4.1+cu118
└── Mixed Precision: ENABLED ✅

🔧 INFERENCE CONFIGURATION:
├── Model: light 3D U-Net
├── Volume Size: (64, 64, 64)
├── Mixed Precision: ✅ ENABLED
├── Test-Time Augmentation: ✅ ENABLED
├── Uncertainty Estimation: ✅ ENABLED
└── Confidence Threshold: 0.5


In [2]:
print(f"\n🧠 LOADING TRAINED MODEL...")

model = create_unet3d(config.model_name).to(device)

try:
    if Path(checkpoint_path).exists():
        print(f"📂 Loading checkpoint: {checkpoint_path}")
        checkpoint = torch.load(checkpoint_path, map_location=device, weights_only=False)
        model.load_state_dict(checkpoint['model_state_dict'])
        best_val_dice = checkpoint.get('best_val_dice', 0.0)
        print(f"✅ Checkpoint loaded! Best validation Dice: {best_val_dice:.4f}")
    elif Path(model_path).exists():
        print(f"📂 Loading model weights: {model_path}")
        model.load_state_dict(torch.load(model_path, map_location=device, weights_only=True))
        print(f"✅ Model weights loaded!")
    else:
        print(f"⚠️  No trained model found. Using randomly initialized weights.")
except Exception as e:
    print(f"❌ Error loading model: {str(e)}")
    print(f"⚠️  Using randomly initialized weights.")

model.eval()

num_params = sum(p.numel() for p in model.parameters())
model_size_mb = num_params * 4 / (1024 * 1024)

print(f"\n✅ MODEL READY FOR INFERENCE:")
print(f"├── Architecture: {config.model_name.upper()} 3D U-Net")
print(f"├── Total parameters: {num_params:,}")
print(f"├── Model size: {model_size_mb:.1f} MB")
print(f"├── Input channels: 4 (T1, T1CE, T2, FLAIR)")
print(f"├── Output channels: 1 (tumor segmentation)")
print(f"└── Device: {next(model.parameters()).device}")

preprocessor = VolumePreprocessor(config)
postprocessor = PostProcessor3D(config)
visualizer = VolumeVisualizer()

print(f"\n🔧 INFERENCE COMPONENTS INITIALIZED:")
print(f"├── Volume Preprocessor: Ready")
print(f"├── Post Processor: Ready")
print(f"├── Visualizer: Ready")
print(f"└── Components: Ready")


🧠 LOADING TRAINED MODEL...
📂 Loading checkpoint: c:\Users\surya\NueroVisionAI\checkpoints\3d\best_model.pth
✅ Checkpoint loaded! Best validation Dice: 0.8974

✅ MODEL READY FOR INFERENCE:
├── Architecture: LIGHT 3D U-Net
├── Total parameters: 1,402,561
├── Model size: 5.4 MB
├── Input channels: 4 (T1, T1CE, T2, FLAIR)
├── Output channels: 1 (tumor segmentation)
└── Device: cuda:0

🔧 INFERENCE COMPONENTS INITIALIZED:
├── Volume Preprocessor: Ready
├── Post Processor: Ready
├── Visualizer: Ready
└── Components: Ready
📂 Loading checkpoint: c:\Users\surya\NueroVisionAI\checkpoints\3d\best_model.pth
✅ Checkpoint loaded! Best validation Dice: 0.8974

✅ MODEL READY FOR INFERENCE:
├── Architecture: LIGHT 3D U-Net
├── Total parameters: 1,402,561
├── Model size: 5.4 MB
├── Input channels: 4 (T1, T1CE, T2, FLAIR)
├── Output channels: 1 (tumor segmentation)
└── Device: cuda:0

🔧 INFERENCE COMPONENTS INITIALIZED:
├── Volume Preprocessor: Ready
├── Post Processor: Ready
├── Visualizer: Ready
└── Co

In [3]:
class InferenceEngine:
    def __init__(self, model, config, preprocessor, postprocessor, use_tta=True, use_uncertainty=True, confidence_threshold=0.5):
        self.model = model
        self.config = config
        self.preprocessor = preprocessor
        self.postprocessor = postprocessor
        self.device = next(model.parameters()).device
        self.use_tta = use_tta
        self.use_uncertainty = use_uncertainty
        self.confidence_threshold = confidence_threshold
        
    @torch.no_grad()
    def predict_single(self, volume_dict, use_tta=True, return_uncertainty=False):
        self.model.eval()
        
        if 'synthetic_volume' in volume_dict:
            input_tensor = torch.from_numpy(volume_dict['synthetic_volume']).unsqueeze(0).float().to(self.device)
        else:
            preprocessed_result = preprocess_for_inference(volume_dict, self.config)
            processed_volume = preprocessed_result['preprocessed_data']
            input_tensor = torch.from_numpy(processed_volume).unsqueeze(0).float().to(self.device)
        
        if use_tta and self.use_tta:
            predictions = []
            
            augmentations = [
                lambda x: x,
                lambda x: torch.flip(x, [2]),
                lambda x: torch.flip(x, [3]),
                lambda x: torch.flip(x, [4]),
                lambda x: torch.flip(x, [2, 3]),
                lambda x: torch.flip(x, [2, 4]),
                lambda x: torch.flip(x, [3, 4]),
                lambda x: torch.flip(x, [2, 3, 4])
            ]
            
            reverse_augmentations = [
                lambda x: x,
                lambda x: torch.flip(x, [2]),
                lambda x: torch.flip(x, [3]),
                lambda x: torch.flip(x, [4]),
                lambda x: torch.flip(x, [2, 3]),
                lambda x: torch.flip(x, [2, 4]),
                lambda x: torch.flip(x, [3, 4]),
                lambda x: torch.flip(x, [2, 3, 4])
            ]
            
            for aug, rev_aug in zip(augmentations, reverse_augmentations):
                augmented_input = aug(input_tensor)
                
                if self.config.mixed_precision:
                    with autocast():
                        pred = self.model(augmented_input)
                else:
                    pred = self.model(augmented_input)
                
                pred = torch.sigmoid(pred)
                pred = rev_aug(pred)
                predictions.append(pred)
            
            prediction_stack = torch.stack(predictions)
            mean_prediction = prediction_stack.mean(dim=0)
            
            if return_uncertainty:
                uncertainty = prediction_stack.std(dim=0)
                return mean_prediction, uncertainty
            else:
                return mean_prediction
        
        else:
            if self.config.mixed_precision:
                with autocast():
                    prediction = self.model(input_tensor)
            else:
                prediction = self.model(input_tensor)
            
            prediction = torch.sigmoid(prediction)
            
            if return_uncertainty:
                return prediction, None
            else:
                return prediction
    
    def batch_inference(self, case_paths, save_results=True):
        results = []
        
        for case_path in tqdm(case_paths, desc="Processing cases"):
            case_id = Path(case_path).stem if isinstance(case_path, (str, Path)) else case_path.get('case_id', 'unknown')
            
            try:
                start_time = time.time()
                
                if isinstance(case_path, dict):
                    volume_dict = case_path
                else:
                    volume_dict = self._load_case_files(case_path)
                
                prediction, uncertainty = self.predict_single(
                    volume_dict, 
                    use_tta=self.use_tta,
                    return_uncertainty=self.use_uncertainty
                )
                
                prediction_np = prediction.squeeze().cpu().numpy()
                uncertainty_np = uncertainty.squeeze().cpu().numpy() if uncertainty is not None else None
                
                binary_mask = (prediction_np > self.confidence_threshold).astype(np.uint8)
                binary_mask = self.postprocessor.postprocess_mask(binary_mask)
                
                inference_time = time.time() - start_time
                
                result = {
                    'case_id': case_id,
                    'prediction_mask': binary_mask,
                    'probability_map': prediction_np,
                    'uncertainty_map': uncertainty_np,
                    'inference_time': inference_time,
                    'volume_shape': prediction_np.shape,
                    'confidence_threshold': self.confidence_threshold
                }
                
                if save_results:
                    self._save_result(result)
                
                results.append(result)
                
                print(f"✅ {case_id}: {inference_time:.2f}s")
                
            except Exception as e:
                print(f"❌ Error processing {case_id}: {str(e)}")
                continue
        
        return results
    
    def _load_case_files(self, case_path):
        case_path = Path(case_path)
        volume_dict = {}
        
        modalities = ['t1n', 't1c', 't2w', 't2f']
        for modality in modalities:
            matching_files = list(case_path.parent.glob(f"*{modality}*.nii.gz"))
            if matching_files:
                volume_dict[modality] = str(matching_files[0])
        
        return volume_dict
    
    def _save_result(self, result):
        output_dir = Path(self.config.output_dir) / "inference_results"
        output_dir.mkdir(parents=True, exist_ok=True)
        
        mask_path = output_dir / f"{result['case_id']}_segmentation.nii.gz"
        prob_path = output_dir / f"{result['case_id']}_probability.nii.gz"
        
        affine = np.eye(4)
        
        mask_nii = nib.Nifti1Image(result['prediction_mask'].astype(np.float32), affine)
        nib.save(mask_nii, mask_path)
        
        prob_nii = nib.Nifti1Image(result['probability_map'], affine)
        nib.save(prob_nii, prob_path)
        
        if result['uncertainty_map'] is not None:
            uncertainty_path = output_dir / f"{result['case_id']}_uncertainty.nii.gz"
            uncertainty_nii = nib.Nifti1Image(result['uncertainty_map'], affine)
            nib.save(uncertainty_nii, uncertainty_path)

inference_engine = InferenceEngine(model, config, preprocessor, postprocessor, use_tta, use_uncertainty, confidence_threshold)

print(f"\n🚀 INFERENCE ENGINE READY:")
print(f"├── Test-Time Augmentation: {'✅' if use_tta else '❌'}")
print(f"├── Uncertainty Estimation: {'✅' if use_uncertainty else '❌'}")
print(f"├── Confidence Threshold: {confidence_threshold}")
print(f"├── Post-processing: Active")
print(f"└── Batch Processing: Ready")


🚀 INFERENCE ENGINE READY:
├── Test-Time Augmentation: ✅
├── Uncertainty Estimation: ✅
├── Confidence Threshold: 0.5
├── Post-processing: Active
└── Batch Processing: Ready


In [4]:
print(f"\n📂 SCANNING FOR TEST CASES...")

test_data_dir = Path(config.data_dir)
test_cases = []

if test_data_dir.exists():
    nii_files = list(test_data_dir.glob("*.nii.gz"))
    
    case_groups = {}
    for file_path in nii_files:
        filename = file_path.stem.replace('.nii', '')
        
        parts = filename.split('-')
        if len(parts) >= 3:
            case_id = '-'.join(parts[:3])
        else:
            case_id = filename.split('_')[0] if '_' in filename else filename
        
        if case_id not in case_groups:
            case_groups[case_id] = {}
        
        modality = None
        for mod in ['t1n', 't1c', 't2w', 't2f', 'seg']:
            if mod in filename.lower():
                modality = mod
                break
        
        if modality and modality != 'seg':
            case_groups[case_id][modality] = str(file_path)
    
    for case_id, modalities in case_groups.items():
        if len(modalities) >= 2:
            modalities['case_id'] = case_id
            test_cases.append(modalities)

print(f"✅ Found {len(test_cases)} test cases:")
for i, case in enumerate(test_cases[:5]):
    modality_count = len([k for k in case.keys() if k != 'case_id'])
    print(f"├── {case['case_id']}: {modality_count} modalities")
if len(test_cases) > 5:
    print(f"└── ... and {len(test_cases) - 5} more cases")

if not test_cases:
    print(f"⚠️  No test cases found. Creating synthetic test case...")
    
    synthetic_volume = torch.randn(4, 64, 64, 64) * 0.1
    
    center = [32, 32, 32]
    for i in range(4):
        for z in range(64):
            for y in range(64):
                for x in range(64):
                    dist = ((z - center[0])**2 + (y - center[1])**2 + (x - center[2])**2) ** 0.5
                    if dist < 25:
                        synthetic_volume[i, z, y, x] += 0.5
    
    tumor_center = [30, 35, 32]
    tumor_size = 8
    for z in range(max(0, tumor_center[0] - tumor_size), min(64, tumor_center[0] + tumor_size)):
        for y in range(max(0, tumor_center[1] - tumor_size), min(64, tumor_center[1] + tumor_size)):
            for x in range(max(0, tumor_center[2] - tumor_size), min(64, tumor_center[2] + tumor_size)):
                dist = ((z - tumor_center[0])**2 + (y - tumor_center[1])**2 + (x - tumor_center[2])**2) ** 0.5
                if dist < tumor_size:
                    for i in range(4):
                        synthetic_volume[i, z, y, x] += 0.4
    
    synthetic_case = {
        'case_id': 'synthetic_test_case',
        'synthetic_volume': synthetic_volume.numpy(),
        'volume_shape': (64, 64, 64)
    }
    test_cases = [synthetic_case]
    print(f"✅ Synthetic test case created")

print(f"\n🎯 INFERENCE CONFIGURATION:")
print(f"├── Test cases: {len(test_cases)}")
print(f"├── Test-Time Augmentation: {'✅' if use_tta else '❌'}")
print(f"├── Uncertainty Estimation: {'✅' if use_uncertainty else '❌'}")
print(f"├── Confidence Threshold: {confidence_threshold}")
print(f"└── Output Directory: {config.output_dir}")


📂 SCANNING FOR TEST CASES...
✅ Found 209 test cases:
├── BraTS-GLI-00002: 2 modalities
├── BraTS-GLI-00014: 3 modalities
├── BraTS-GLI-00016: 3 modalities
├── BraTS-GLI-00088: 2 modalities
├── BraTS-GLI-00090: 2 modalities
└── ... and 204 more cases

🎯 INFERENCE CONFIGURATION:
├── Test cases: 209
├── Test-Time Augmentation: ✅
├── Uncertainty Estimation: ✅
├── Confidence Threshold: 0.5
└── Output Directory: c:\Users\surya\NueroVisionAI\outputs\3d


In [5]:
inference_results = []
total_inference_time = 0

for idx, test_case in enumerate(test_cases, 1):
    print(f"\n🧠 Processing Case {idx}/{len(test_cases)}: {test_case['case_id']}")
    print("─" * 60)
    try:
        start_time = time.time()
        
        volume_paths = {k: v for k, v in test_case.items() if k != 'case_id'}
        prediction, uncertainty = inference_engine.predict_single(
            volume_paths,
            use_tta=use_tta,
            return_uncertainty=use_uncertainty
        )
        
        prediction_np = prediction.squeeze().cpu().numpy()
        uncertainty_np = uncertainty.squeeze().cpu().numpy() if uncertainty is not None else None
        
        binary_mask = (prediction_np > confidence_threshold).astype(np.uint8)
        binary_mask = postprocessor.postprocess_mask(binary_mask)
        
        inference_time = time.time() - start_time
        total_inference_time += inference_time
        
        result = {
            'case_id': test_case['case_id'],
            'prediction_mask': binary_mask,
            'probability_map': prediction_np,
            'uncertainty_map': uncertainty_np,
            'inference_time': inference_time,
            'volume_shape': prediction_np.shape,
            'confidence_threshold': confidence_threshold
        }
        
        inference_results.append(result)
        
        tumor_volume = np.sum(binary_mask)
        max_prob = np.max(prediction_np)
        
        print(f"✅ Inference completed in {inference_time:.2f}s")
        print(f"├── Tumor detected: {'Yes' if tumor_volume > 0 else 'No'}")
        print(f"├── Tumor volume: {tumor_volume} voxels")
        print(f"└── Max probability: {max_prob:.4f}")
        
    except Exception as e:
        print(f"❌ Error processing {test_case['case_id']}: {e}")
        continue

print(f"\n🎯 INFERENCE SUMMARY:")
print(f"├── Total cases processed: {len(inference_results)}")
print(f"├── Total processing time: {format_time(total_inference_time)}")
print(f"└── Average time per case: {total_inference_time/max(len(inference_results), 1):.2f}s")


🧠 Processing Case 1/209: BraTS-GLI-00002
────────────────────────────────────────────────────────────
❌ Error processing BraTS-GLI-00002: 'preprocessed_data'

🧠 Processing Case 2/209: BraTS-GLI-00014
────────────────────────────────────────────────────────────
❌ Error processing BraTS-GLI-00002: 'preprocessed_data'

🧠 Processing Case 2/209: BraTS-GLI-00014
────────────────────────────────────────────────────────────
❌ Error processing BraTS-GLI-00014: 'preprocessed_data'

🧠 Processing Case 3/209: BraTS-GLI-00016
────────────────────────────────────────────────────────────
❌ Error processing BraTS-GLI-00014: 'preprocessed_data'

🧠 Processing Case 3/209: BraTS-GLI-00016
────────────────────────────────────────────────────────────
❌ Error processing BraTS-GLI-00016: 'preprocessed_data'

🧠 Processing Case 4/209: BraTS-GLI-00088
────────────────────────────────────────────────────────────
❌ Error processing BraTS-GLI-00088: 'preprocessed_data'

🧠 Processing Case 5/209: BraTS-GLI-00090
────

In [6]:
if inference_results:
    print(f"\n📊 VISUALIZING INFERENCE RESULTS...")
    
    for i, result in enumerate(inference_results[:3]):
        case_id = result['case_id']
        prediction_mask = result['prediction_mask']
        probability_map = result['probability_map']
        
        print(f"\n🧠 Visualizing Case: {case_id}")
        
        fig, axes = plt.subplots(2, 3, figsize=(15, 10))
        fig.suptitle(f'3D Brain Tumor Segmentation Results: {case_id}', fontsize=16, fontweight='bold')
        
        mid_slice = prediction_mask.shape[2] // 2
        
        slice_indices = [
            max(0, mid_slice - 10),
            mid_slice,
            min(prediction_mask.shape[2] - 1, mid_slice + 10)
        ]
        
        for j, slice_idx in enumerate(slice_indices):
            prob_slice = probability_map[:, :, slice_idx]
            mask_slice = prediction_mask[:, :, slice_idx]
            
            axes[0, j].imshow(prob_slice, cmap='hot', vmin=0, vmax=1)
            axes[0, j].set_title(f'Probability Map - Slice {slice_idx}', fontweight='bold')
            axes[0, j].axis('off')
            
            overlay = np.zeros((*mask_slice.shape, 3))
            overlay[mask_slice > 0] = [1, 0, 0]
            
            axes[1, j].imshow(prob_slice, cmap='gray', alpha=0.7)
            axes[1, j].imshow(overlay, alpha=0.5)
            axes[1, j].set_title(f'Segmentation Overlay - Slice {slice_idx}', fontweight='bold')
            axes[1, j].axis('off')
        
        plt.tight_layout()
        
        if len(inference_results) <= 3:
            plt.show()
        else:
            output_vis_dir = Path(config.output_dir) / "visualizations"
            output_vis_dir.mkdir(parents=True, exist_ok=True)
            plot_path = output_vis_dir / f"{case_id}_segmentation_results.png"
            plt.savefig(plot_path, dpi=300, bbox_inches='tight')
            plt.close()
            print(f"📊 Visualization saved: {plot_path}")

print(f"\n📈 GENERATING INFERENCE SUMMARY REPORT...")

summary_report = {
    'inference_summary': {
        'total_cases': len(test_cases),
        'successful_cases': len(inference_results),
        'failed_cases': len(test_cases) - len(inference_results),
        'total_processing_time_seconds': total_inference_time,
        'average_time_per_case_seconds': total_inference_time / max(len(inference_results), 1),
        'config': {
            'model_name': config.model_name,
            'input_size': config.input_size,
            'use_tta': use_tta,
            'use_uncertainty': use_uncertainty,
            'confidence_threshold': confidence_threshold,
            'mixed_precision': config.mixed_precision
        }
    },
    'case_results': []
}

for result in inference_results:
    tumor_volume = np.sum(result['prediction_mask'])
    tumor_percentage = (tumor_volume / np.prod(result['prediction_mask'].shape)) * 100
    
    case_summary = {
        'case_id': result['case_id'],
        'inference_time_seconds': result['inference_time'],
        'tumor_detected': tumor_volume > 0,
        'tumor_volume_voxels': int(tumor_volume),
        'tumor_percentage': round(tumor_percentage, 2),
        'max_probability': round(float(np.max(result['probability_map'])), 4),
        'mean_tumor_probability': round(float(np.mean(result['probability_map'][result['prediction_mask'] > 0])), 4) if tumor_volume > 0 else 0,
        'volume_shape': result['prediction_mask'].shape,
        'confidence_threshold': result['confidence_threshold']
    }
    
    if result.get('uncertainty_map') is not None:
        case_summary['mean_uncertainty'] = round(float(np.mean(result['uncertainty_map'][result['prediction_mask'] > 0])), 4) if tumor_volume > 0 else 0
    
    summary_report['case_results'].append(case_summary)

report_output_dir = Path(config.output_dir)
report_output_dir.mkdir(parents=True, exist_ok=True)

report_path = report_output_dir / "inference_summary_report.json"
with open(report_path, 'w') as f:
    json.dump(summary_report, f, indent=2)

print(f"✅ Inference summary report saved: {report_path}")

print(f"\n📋 FINAL INFERENCE REPORT:")
print("=" * 80)
tumor_cases = [r for r in inference_results if np.sum(r['prediction_mask']) > 0]
print(f"🔍 DETECTION SUMMARY:")
print(f"├── Cases processed: {len(inference_results)}")
print(f"├── Tumors detected: {len(tumor_cases)}")
print(f"├── Detection rate: {len(tumor_cases)/max(len(inference_results), 1)*100:.1f}%")
print(f"└── False negative rate: {(len(inference_results)-len(tumor_cases))/max(len(inference_results), 1)*100:.1f}%")

if tumor_cases:
    volumes = [np.sum(r['prediction_mask']) for r in tumor_cases]
    probabilities = [np.max(r['probability_map']) for r in tumor_cases]
    
    print(f"\n📊 TUMOR CHARACTERISTICS:")
    print(f"├── Average volume: {np.mean(volumes):.0f} voxels")
    print(f"├── Volume range: {np.min(volumes):.0f} - {np.max(volumes):.0f} voxels")
    print(f"├── Average max probability: {np.mean(probabilities):.4f}")
    print(f"└── Probability range: {np.min(probabilities):.4f} - {np.max(probabilities):.4f}")

print(f"\n⚡ PERFORMANCE METRICS:")
print(f"├── Total processing time: {format_time(total_inference_time)}")
print(f"├── Average time per case: {total_inference_time/max(len(inference_results), 1):.2f}s")
if total_inference_time > 0:
    print(f"├── Throughput: {len(inference_results)/total_inference_time*60:.1f} cases/minute")
else:
    print(f"├── Throughput: N/A (no processing time recorded)")
print(f"└── Model parameters: {num_params:,}")

print(f"\n🎉 3D BRAIN TUMOR SEGMENTATION INFERENCE COMPLETE!")
print(f"🔥 PRODUCTION-READY | GPU ACCELERATED | UNCERTAINTY AWARE")
print(f"💾 Results saved to: {config.output_dir}")
print("=" * 80)


📈 GENERATING INFERENCE SUMMARY REPORT...
✅ Inference summary report saved: c:\Users\surya\NueroVisionAI\outputs\3d\inference_summary_report.json

📋 FINAL INFERENCE REPORT:
🔍 DETECTION SUMMARY:
├── Cases processed: 0
├── Tumors detected: 0
├── Detection rate: 0.0%
└── False negative rate: 0.0%

⚡ PERFORMANCE METRICS:
├── Total processing time: 0.0s
├── Average time per case: 0.00s
├── Throughput: N/A (no processing time recorded)
└── Model parameters: 1,402,561

🎉 3D BRAIN TUMOR SEGMENTATION INFERENCE COMPLETE!
🔥 PRODUCTION-READY | GPU ACCELERATED | UNCERTAINTY AWARE
💾 Results saved to: c:\Users\surya\NueroVisionAI\outputs\3d
